In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 30 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.


# DATASET INFO
NUM_CLASSES = 2 # set the number of classes in your dataset
DATA_DIR = '/content/drive/My Drive/progetto/MAMMO_MBONLY' # to run with the sample dataset, just set to 'hymenoptera_data'

# DATALOADER PROPERTIES
BATCH_SIZE = 10 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
#CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)
GPU_MODE = 1 # set to 1 if want to run on gpu.

use_gpu = 0



Using device: cuda


In [ ]:
# If you want to read more, transforms is a function from torchvision, and you can go read more here - http://pytorch.org/docs/master/torchvision/transforms.html
data_transforms = {
    'train': transforms.Compose([
        #transforms.ToPILImage(),
        #transforms.Resize(224),
        transforms.RandomResizedCrop(224),
        #transforms.Grayscale(3),
        #transforms.ColorJitter( contrast=0.5),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.ToPILImage(),                        
        #transforms.RandomResizedCrop(224),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        #transforms.ColorJitter(brightness=0, contrast=1, saturation=0.7, hue=0),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
data_dir = DATA_DIR
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=0
                                               )
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

In [ ]:
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# Run the functions and save the best model in the function model_ft.
num_epochs = 30
#TRAINING GALASSO

#RESNET
# model = models.resnet50(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, NUM_CLASSES)
# model.to(device)

# #LOAD A MODEL
checkpoint = torch.load('/content/drive/My Drive/progetto/mask_rectangle_best_resnet50.pth')
#model = torch.load('/content/drive/My Drive/progetto/TOMO_resnet50.pth')
model = models.resnet50(pretrained=True)    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)  # make the change
model.load_state_dict(checkpoint)
model.to(device)

#VGG-16

# model = models.vgg16_bn(pretrained=False)
# num_features = model.classifier[6].in_features
# features = list(model.classifier.children())[:-1] # Remove last layer
# features.extend([nn.Linear(num_features, NUM_CLASSES)]) # Add our layer with 4 outputs
# model.classifier = nn.Sequential(*features)
# model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=DECAY_WEIGHT)
early_epoch = 0 
# Train the model
lr = 0.0001
total_step = len(dset_loaders['train'])
loss_train = []
loss_val = []
best_accuracy = None
accuracy_val = []
best_model = model # get a new instance
for epoch in range(num_epochs):
    print('Epoch:', epoch+1)
    
    model.train()

    loss_iter = 0
    for i, (images, labels) in enumerate(dset_loaders['train']):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_iter += loss.item()

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    loss_train.append(loss_iter/(len(dset_loaders['train'])*BATCH_SIZE))


    # Code to update the lr
    lr *= 0.99
    update_lr(optimizer, lr)
    
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        loss_iter = 0
        for images, labels in dset_loaders['val']:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss = criterion(outputs, labels)
            loss_iter += loss.item()
            
        loss_val.append(loss_iter/(len(dset_loaders['val'])*BATCH_SIZE))

        accuracy = 100 * correct / total
        accuracy_val.append(accuracy)
        
        print('Validataion accuracy is: {} %'.format(accuracy))
        #################################################################################
        # TODO: Q2.b Use the early stopping mechanism from previous questions to save   #
        # the model with the best validation accuracy so-far (use best_model).          #
        #################################################################################

        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        if epoch >=3 and accuracy > max(accuracy_val[:-1]):
            best_model = copy.deepcopy(model)
            early_epoch = epoch
        
        if epoch == num_epochs - 1:
            print(f'\nEarly Stopping at the epoch: {early_epoch+1} with accuracy: {accuracy_val[early_epoch]:.2f} and acc: {100 * accuracy_val[early_epoch]:.2f}%')


Epoch: 1
Epoch [1/30], Step [10/18], Loss: 0.6320
Validataion accuracy is: 66.17647058823529 %
Epoch: 2
Epoch [2/30], Step [10/18], Loss: 0.7516
Validataion accuracy is: 44.11764705882353 %
Epoch: 3
Epoch [3/30], Step [10/18], Loss: 0.6398
Validataion accuracy is: 47.05882352941177 %
Epoch: 4
Epoch [4/30], Step [10/18], Loss: 0.5939
Validataion accuracy is: 45.588235294117645 %
Epoch: 5
Epoch [5/30], Step [10/18], Loss: 0.7881
Validataion accuracy is: 45.588235294117645 %
Epoch: 6
Epoch [6/30], Step [10/18], Loss: 0.9301
Validataion accuracy is: 36.76470588235294 %
Epoch: 7
Epoch [7/30], Step [10/18], Loss: 0.6637
Validataion accuracy is: 35.294117647058826 %
Epoch: 8
Epoch [8/30], Step [10/18], Loss: 0.6466
Validataion accuracy is: 63.23529411764706 %
Epoch: 9
Epoch [9/30], Step [10/18], Loss: 0.5908
Validataion accuracy is: 57.35294117647059 %
Epoch: 10
Epoch [10/30], Step [10/18], Loss: 0.5307
Validataion accuracy is: 67.6470588235294 %
Epoch: 11
Epoch [11/30], Step [10/18], Loss: 0

In [ ]:
for data in dset_loaders['val']:
  image,lab = data
  print(lab)


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0])


In [ ]:
#filename_pth = '/content/drive/My Drive/progetto/Mammo_poli_resnet_notpretrained.pth'
#torch.save(model.state_dict(), filename_pth)

import pandas as pd
df = pd.DataFrame({'epoch' : range(len(loss_train)), 
                       'loss_train':loss_train,
                       'loss_val':loss_val,
                       'accuracy_val': accuracy_val})
    
df.to_csv('/content/drive/My Drive/progetto/Mammo_poli_transfer_data.csv')

In [ ]:
# If you want to read more, transforms is a function from torchvision, and you can go read more here - http://pytorch.org/docs/master/torchvision/transforms.html
data_transforms_test = {
    'val': transforms.Compose([
        #transforms.ToPILImage(),
        transforms.RandomResizedCrop(224),
        #transforms.ColorJitter(brightness=0, contrast=1, saturation=0.7, hue=0),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
   
}

In [ ]:
data_dir = '/content/drive/My Drive/progetto/TOMO_train_val'
dsets = {'val': datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms_test['val'])
         }
dset_loaders_test = {'val': torch.utils.data.DataLoader(dsets['val'], batch_size=120,
                                               shuffle=True, num_workers=0
                                               )}
              
#dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
#dset_classes = dsets['t'].classes

In [ ]:
model = torch.load('/content/drive/My Drive/progetto/TOMO_resnet50.pth')
#model = models.resnet50(pretrained=True)    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)  # make the change
# model.load_state_dict(checkpoint)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
with torch.no_grad():
        correct = 0
        total = 0
        loss_iter = 0
        for images, labels in dset_loaders_test['val']:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

In [ ]:
correct

52

In [ ]:
acc = 100 * correct / total
print(acc)

43.333333333333336
